# LIXO 
---

OCUPANTES

In [ ]:
async def run(self):
            '''
            se saida-> vai para saida
            se não random
            '''

            if self.clossest_exit()!= -1:
                EvacuationUI.update_agent_position(self.agent.name,self.clossest_exit())

            if self.agent.exits["Exit 1"] == 'closed':
                print(f"{self.agent.name} found Exit 1 clsoed, rerouting to another exit...")

            else:
                print(f"{self.agent.name} is heading to Exit 1")

            if self.agent.elevator == 'off':
                print(f"{self.agent.name} found the elevator off, will try to use stairs")


            await asyncio.sleep(2)

In [ ]:
async def clossest_exit(self):
            
            #get poss of ocupant based on name / name==poss na list
            poss_exits = EvacuationUI.exits_doors()

            ###ver caso não existam saídas(saida==curr_poss ou -1)-> andar aleatóriamente
            if not poss_exits:return -1

            x, y = EvacuationUI.occupants_loc[self.name] #[x, y]

            #ver prioridade -> mais á frente... :)

            dist = EvacuationUI.grid_size + 1
            i = 0
            p_dist = -1

            for i in range(poss_exits):
                x1 = poss_exits[i][0]
                y1 = poss_exits[i][1]
                if np.sqrt((x-x1)**2 + (y-y1)**2)<dist:
                    dist = np.sqrt((x-x1)**2 + (y-y1)**2)
                    p_dist = i

            return poss_exits[p_dist][0], poss_exits[p_dist][1]

        def find_path(self, target):
            
            
            open_set = []
            position = self.environment.get_occupant_loc(self.jid)
            heapq.heappush(open_set, (0, position))  # Priority queue with (cost, position)
            came_from = {}
            g_score = {position: 0}
            f_score = {position: self.heuristic(position, target)}

            while open_set:
                _, current = heapq.heappop(open_set)

                # Exit reached
                if current == target:  
                    #o occ guarda o caminho q quer fazer 
                    self.path = self.reconstruct_path(came_from, current)
                    return self.path



                x, y = current
                neighbors = self.astar_possible_moves(x, y) #where he can walk to(they are poss moves)
                for nx, ny in neighbors:
                    tentative_g_score = g_score[current] + 1
                    if tentative_g_score < g_score.get((nx, ny), float('inf')):
                        came_from[(nx, ny)] = current
                        g_score[(nx, ny)] = tentative_g_score
                        f_score[(nx, ny)] = tentative_g_score + self.heuristic((nx, ny), target)

                        # Avoid duplicates in the open_set
                        if (nx, ny) not in f_score:
                            heapq.heappush(open_set, (f_score[(nx, ny)], (nx, ny)))

            return None  # No path found

        def heuristic(self, a, b):
            
            return np.sqrt((a[0]-b[0])**2 + (a[1]-b[1])**2)

        def reconstruct_path(self, came_from, current):
            
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.reverse()
            return path


    class CheckForHealthState(CyclicBehaviour):
        async def run(self):
            while(not self.agent.busy):
                await asyncio.sleep(1)
            await self.ask_health_state()
            await asyncio.sleep(10)

        async def ask_health_state(self):
            num_occupants = self.agent.environment.num_occupants
            for i in range(num_occupants):
                id = f"occupant{i}@localhost"
                if str(id) in self.agent.environment.occupants_loc.keys():
                    print(f"ER Agent {self.agent.jid} ks sending message to {id}")
                    msg = Message(to=f"occupant{i}@localhost")
                    msg.set_metadata("performative", "informative")
                    msg.body = "[ER] Please give me information on your health state"

                    await self.send(msg)


    class ReceiveHealthState(CyclicBehaviour):
        async def run(self):
            to_help_list = []
            while True:  # Continuously listen for messages
                await self.receive_health_state(to_help_list)
                #[id, healf, x, y, z]


        async def receive_health_state(self, to_help_list):
            '''
            se occ tiver no pick da health, não pede ajuda, só foge
            '''
            msg = await self.receive(timeout=10)  # Wait for a message with a 10-second timeout

            if msg:
                # Assuming msg.body contains the message text
                content = msg.body  # or msg.content, depending on the message library

                # Split the message by semicolons to isolate sections
                parts = content.split(";")

                try:
                    # Extract `id`
                    id_part = parts[0].split(":")[1].strip()

                    # Extract `position` (x, y, z) - splitting by `:` and `,`
                    position_part = parts[1].split(":")[1].strip()
                    x, y, z = map(int, position_part.strip("()").split(","))

                    # Extract `health state`
                    health_state_part = parts[2].split(":")[1].strip()
                    health_state = int(health_state_part)

                    # Create the array with agent data
                    occ = [id_part, health_state, x, y, z]

                    print(f"Agent data Received by ER Agent:\n - Id: {occ[0]};\n - Health State: {occ[1]};\n - Position:{occ[2],occ[3],occ[4]}")
                    to_help_list.append(occ)

                    if health_state==1: #agent é curável
                        cure_behaviour = self.Cure(occ, to_help_list)
                        self.agent.add_behaviour(cure_behaviour)

                except IndexError as e:
                    print("Failed to parse message. Make sure the message format is correct:", e)
                except ValueError as e:
                    print("Failed to convert data to integers. Check the data format:", e)



        # se ff -> o occ já foi visto por um médico(não sofre dano ao longo do tempo)
        async def clear_path(self, vetor):
            '''
            se houver um problema de dimenções pequenas eles podem fazer com que ele desapareça
            tem q ser no msm andar

            a função é chamada quando se encontra fogo no caminho

            recebe o vetor da direção, e vê 7 casas à frente e só apaga o fogo se houver pelo menos
            um quadrado não em chamas(apaga enguanto passa)
            '''
            #vetor = [1, -1] exemplo
            x, y, z = self.agent.environment.get_er_loc(self.agent.jid)
            can = 1
            while [x+can*vetor[0], y+can*vetor[1], z] in self.agent.environment.obstacles[x+can*vetor[0], y+can*vetor[1], z]:
                can+=1
            if can!=7:
                self.make_wave(vetor)

        async def make_wave(self, vector):

            x, y, z = self.agent.environment.get_er_loc(self.agent.jid)

            can = 1
            while [x+can*vector[0], y+can*vector[1], z] in self.agent.environment.obstacles[x+can*vector[0], y+can*vector[1], z]:
                can+=1
                self.agent.environment.obstacles.pop(x+can*vector[0], y+can*vector[1], z)

            pass


---

BUILDING_AGENT

----

EMERGENCY_RESPONDER

---